## Tensorflow
- `Tensorflow`가 예전에는 쓰기 어려운 모델이었음 (코딩할 줄 아는 사람들만 사용)
- 그래서 `pytorch`가 많이 쓰이다 보니, `Tensorflow`에서도 쉽게 사용할 수 있는 `Keras` 만듦
- `Tensorflow 2.0`에서는 `keras`와 합쳐진 `tf.keras.Model`이나 `Sequential` 많이 사용
- `Tensorflow`에서 train step, test step을 사용하는 class 구조는 `pytorch lightening`과 비슷
  - `pytorch lightening` : `pytorch`를 더 쉽게 사용하기 위한 library
- Optimizer : Tensorflow addon
  - https://www.tensorflow.org/addons/overview?hl=ko
  - https://github.com/tensorflow/addons
    - 여러 tensorflow 개발자들이 다양한 optimizer 구현 코드 업로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')
drive_project_root = '/content/drive/MyDrive/#fastcampus'
# !pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
pip install tensorflow_addons

In [ ]:
pip install wandb

In [ ]:
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa

import wandb

GPU 확인

In [ ]:
tf.config.list_physical_devices()

In [ ]:
!nvidia-smi

https://www.tensorflow.org/</br>
- https://www.tensorflow.org/overview/?hl=ko</br>
- 튜토리얼 : https://www.tensorflow.org/tutorials?hl=ko
- API > Tensorflow : 각 함수에 대한 설명
  - 구글에 'Tensorflow API 한글' 검색하면 번역본도 볼 수 있음

초보자용 vs 전문가용
- 수업에서는 전문가용으로 할 예정
- 초보자용에서 사용하는 Sequential 버전(순차적으로 build 하는 방법)에는 한계가 있기 때문
- 실제 현업/연구에서는 Sequential 거의 안 씀

## define gpu
- https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy
- This strategy is typically used for training on one machine with multiple GPUs.
- 아래 코드 결과 보면 GPU 0번 잡아서 가져옴

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

## data, data loader 정의

- 사실 tensorflow에서는 data loader를 정의를 안 하기도 함
- dataset으로 그냥 처리 가능
- 단, 여기서는 pytorch 방식과 비교하기 위해 사용함

In [ ]:
with mirrored_strategy.scope():

    # dataset 정의 =====================================================================
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # normalization
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # train/val splits
    train_size = int(len(x_train)*0.9)
    val_size = len(x_train) - train_size

    # train, test dataset 정의
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(buffer_size=1024)

    # train dataset을 train과 validation으로 나누기
    # tensorflow에서는 아래와 같이 take, skip 사용해서 데이터 많이 나눔
    train_dataset = dataset.take(train_size)   # train은 dataset에서 train_size만큼 take하고
    val_dataset = dataset.skip(train_size)     # val은 전체 dataset에서 train_size만큼 skip하고 남은 것

    # 검증
    print(f'train total : {len(dataset)} (train : {len(train_dataset)}, validation : {len(val_dataset)})')
    print(f'test : {len(test_dataset)}')

    # dataloader 정의 ==================================================================
    train_batch_size = 100
    val_batch_size = 10
    test_batch_size = 100

    # drop_remainder=True : memory size가 안 맞으면 error 나는 것 방지 (pytorch는 이런 것 자동으로 처리함 = tensorflow와 차이점)
    train_dataloader = train_dataset.batch(train_batch_size, drop_remainder=True)
    val_dataloader = val_dataset.batch(val_batch_size, drop_remainder=True)
    test_dataloader = test_dataset.batch(test_batch_size, drop_remainder=True)

In [ ]:
sample_example = next(iter(train_dataloader))
print(sample_example)

## plot figure

In [ ]:
plt.figure(figsize=(10,10))
for c in range(16):
    plt.subplot(4, 4, c+1)
    plt.imshow(x_train[c].reshape(28,28), cmap='gray')
plt.show()

## make model

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self, input_dim: int, h1_dim: int, h2_dim: int, out_dim: int):
        super().__init__()

        # tensorflow는 pytorch와 다르게 flatten을 하지 않아도 되지만
        # pytorch와 비슷한 구조로 코딩하기 위해 여기서는 썼음
        self.flatten = tf.keras.layers.Flatten()

        # tf nn module vs keras module
        # 1) nn module : tensorflow 1.0에서 사용, 기능이 조금 더 많음
        # 2) keras : tensorflow 2.0에서 사용
        self.linear1 = tf.keras.layers.Dense(input_dim=input_dim, units=h1_dim)
        # self.linear2 = tf.keras.layers.Dense(input_dim=h1_dim, units=h2_dim)
        # -> 이렇게 써도 되지만, pytorh보다 keras는 flexibility가 있어서 input_dim 생략해도 알아서 인지함
        self.linear2 = tf.keras.layers.Dense(units=h2_dim)
        self.linear3 = tf.keras.layers.Dense(units=out_dim)
        self.relu = tf.nn.relu
    
    # tensorflow에서는 'training=Fasle' 구문 꼭 넣기를 권장함
    # 나중에 regularization에서 drop out 할 때 이 부분을 조절할 수 있어야 함
    # - 학습일 때는 켜고, evaluation 때는 끄고
    def call(self, input, training=False):
        x = self.flatten(input)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        out = tf.nn.softmax(out)  # output을 확률값으로 바꿈
        return out
    
    # GradientTape() 구현
    # 따로 구현해도 되지만 class 안에 이렇게 넣어주면 나중에 더 코드가 깔끔해짐
    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"loss": loss})
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"test_loss": loss})
        return logs

### Dropout

In [ ]:
# tf.keras.Model 대신 MLP 써도 됨
# 그러면 위에서 썼던 MLP class를 가져오는 것이니
# MLP 모델과 달라진 __init__, call만 코드 써서 일부 수정해 주고
# 그대로 써도 되는 train_step, test_step는 생략 가능
class MLPWithDropout(tf.keras.Model):
    def __init__(self, input_dim: int, h1_dim: int, h2_dim: int, out_dim: int, dropout_prob: float):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.linear1 = tf.keras.layers.Dense(input_dim=input_dim, units=h1_dim)
        self.linear2 = tf.keras.layers.Dense(units=h2_dim)
        self.linear3 = tf.keras.layers.Dense(units=out_dim)
        self.dropout = tf.keras.layers.Dropout(dropout_prob)
        self.relu = tf.nn.relu
    
    def call(self, input, training=False):
        x = self.flatten(input)
        x = self.relu(self.linear1(x))
        x = self.dropout(x, training=training)
        x = self.relu(self.linear2(x))
        x = self.dropout(x, training=training)
        out = self.linear3(x)
        out = tf.nn.softmax(out)
        return out

    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs     

## Learning Rate scheduler 구현
- A Learning rate schedule is a predefined framework that adjusts the learning rate between epochs or iterations as the training progresses
- tensorflow 페이지 들어가보면 schedules에서 learning rate decay 관련 자료는 많지만 warmup은 많이 쓰임에도 불구하고 자료가 잘 없음
  - 당시 library가 2015년 이전에 많이 정립이 되었는데
  - warmup은 residual network, transformer에 사용하기 위해 2015년 이후에 발견되었기 때문
- 그래서 이번 연습에서는 warmup scheduler를 구현해 볼 예정
- warmup은 맨 처음에 0에 가까운 숫자에서 시작하고 원하는 learning rate까지 linear하게 점점 증가하는 형태
  - 따라서 `def __init__(self, initial_learning_rate)`로 보통은 쓰지만 여기서는 `initial_learning_rate` 부분이 필요하지 않음 

In [ ]:
class LinearWarmupLRScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, lr_peak: float, warmup_end_steps: int,):
        super().__init__()
        self.lr_peak = lr_peak
        self.warmup_end_steps = warmup_end_steps
    
    def __call__(self, step):

        # tensor 형태로 변환 안 하면 error 나기도 해서 tensor 형태로 바꾸기
        step_float = tf.cast(step, tf.float32)
        warmup_end_step = tf.cast(self.warmup_end_steps, tf.float32)
        lr_peak = tf.cast(self.lr_peak, tf.float32)

        # tensor 형태는 if 구문 못 쓰니 tf.cond 형식으로 구현해야 함
        # tf.cond(condition, condition이 true인 경우 값, false인 경우 값)
        return tf.cond(
            step_float < warmup_end_step,
            # if True : 보통 lambda 형태로 많이 구현 (callable 형태) 
            # 만약 step이 0부터 시작하면 lr_peak 값이 0이 되는데, 그러면 문제가 생길 수 있으니 step이 0이 되지 않도록 maximum 처리
            lambda: lr_peak * (tf.math.maximum(step_float, 1) / warmup_end_step),   # -> 서서히 증가할 예정
            # else
            lambda: lr_peak,
        )

잘 만들어졌는지 test
- 결과 : <tf.Tensor: shape=(), dtype=float32, numpy=0.0005>
  - 0.001의 절반에 해당하는 값(0.0005)이 잘 나옴

In [ ]:
a = LinearWarmupLRScheduler(lr_peak=1e-3, warmup_end_steps=1000)
a(500)

In [ ]:
a = LinearWarmupLRScheduler(lr_peak=1e-3, warmup_end_steps=1000)
a(1500)

0을 넣어도 maximum 처리를 해줬기 때문에 0은 아닌 값으로 결과가 잘 나옴

In [ ]:
a = LinearWarmupLRScheduler(lr_peak=1e-3, warmup_end_steps=1000)
a(0)

## AdamP import

아래에서 AdamP 사용하려는데 error 나서 google에 AdamP 검색하고 필요한 코드 부분 그대로 가져와서 붙여 넣음
- https://github.com/taki0112/AdamP-Tensorflow/blob/master/adamp_tf.py
- class AdamP 부분 그대로 붙여 넣으면 코드 오류 남
  - optimizer_v2.OptimizerV2 -> tensorflow.python.keras.optimizer_v2.OptimizerV2
  - github 코드 확인해보면 `optimizer_v2.OptimizerV2`를 `tensorflow.python.keras`에서 import 하는 것 볼 수 있음
  - 그래도 오류남
  - 오래된 문서고 수정된 버전을 안 올려나서 그럴 수 있음
- 이럴 땐 공신력 있는 addon에서 rectified adam import 하는 코드 참고
  - https://github.com/tensorflow/addons/blob/v0.15.0/tensorflow_addons/optimizers/rectified_adam.py#L24-L328
  - 앞에 `@tf.keras.utils.register_keras_serializable(package="Addons")` 붙일 것
  - 안에 `tf.keras.optimizers.Optimizer`로 바꿔볼 것
  - 에러 해결

In [ ]:
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend_config
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops

In [ ]:
#@tf.keras.utils.register_keras_serializable(package="Addons")
class AdamP(tf.keras.optimizers.Optimizer):
    # 출처 적기
    # Code is from https://github.com/taki0112/AdamP-Tensorflow/blob/master/adamp_tf.py with modification
    _HAS_AGGREGATE_GRAD = True

    def __init__(self,
                 learning_rate=0.001,
                 beta_1=0.9,
                 beta_2=0.999,
                 epsilon=1e-8,
                 weight_decay=0.0,
                 delta=0.1, wd_ratio=0.1, nesterov=False,
                 name='AdamP',
                 **kwargs):

        super(AdamP, self).__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
        self._set_hyper('beta_1', beta_1)
        self._set_hyper('beta_2', beta_2)
        self._set_hyper('delta', delta)
        self._set_hyper('wd_ratio', wd_ratio)

        self.epsilon = epsilon or backend_config.epsilon()
        self.weight_decay = weight_decay
        self.nesterov = nesterov

    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        # Separate for-loops to respect the ordering of slot variables from v1.
        for var in var_list:
            self.add_slot(var, 'm')
        for var in var_list:
            self.add_slot(var, 'v')
        for var in var_list:
            self.add_slot(var, 'p')

    def _prepare_local(self, var_device, var_dtype, apply_state):
        super(AdamP, self)._prepare_local(var_device, var_dtype, apply_state)

        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_t = array_ops.identity(self._get_hyper('beta_1', var_dtype))
        beta_2_t = array_ops.identity(self._get_hyper('beta_2', var_dtype))
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)

        lr = apply_state[(var_device, var_dtype)]['lr_t']
        bias_correction1 = 1 - beta_1_power
        bias_correction2 = 1 - beta_2_power

        delta = array_ops.identity(self._get_hyper('delta', var_dtype))
        wd_ratio = array_ops.identity(self._get_hyper('wd_ratio', var_dtype))

        apply_state[(var_device, var_dtype)].update(
            dict(
                lr=lr,
                epsilon=ops.convert_to_tensor_v2(self.epsilon, var_dtype),
                weight_decay=ops.convert_to_tensor_v2(self.weight_decay, var_dtype),
                beta_1_t=beta_1_t,
                beta_1_power=beta_1_power,
                one_minus_beta_1_t=1 - beta_1_t,
                beta_2_t=beta_2_t,
                beta_2_power=beta_2_power,
                one_minus_beta_2_t=1 - beta_2_t,
                bias_correction1=bias_correction1,
                bias_correction2=bias_correction2,
                delta=delta,
                wd_ratio=wd_ratio))

    def set_weights(self, weights):
        params = self.weights
        # If the weights are generated by Keras V1 optimizer, it includes vhats
        # optimizer has 2x + 1 variables. Filter vhats out for compatibility.
        num_vars = int((len(params) - 1) / 2)
        if len(weights) == 3 * num_vars + 1:
            weights = weights[:len(params)]
        super(AdamP, self).set_weights(weights)

    def _resource_apply_dense(self, grad, var, apply_state=None):
        var_device, var_dtype = var.device, var.dtype.base_dtype
        coefficients = ((apply_state or {}).get((var_device, var_dtype))
                        or self._fallback_apply_state(var_device, var_dtype))

        # m_t = beta1 * m + (1 - beta1) * g_t
        m = self.get_slot(var, 'm')
        m_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
        m_t = state_ops.assign(m, m * coefficients['beta_1_t'] + m_scaled_g_values, use_locking=self._use_locking)

        # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
        v = self.get_slot(var, 'v')
        v_scaled_g_values = (grad * grad) * coefficients['one_minus_beta_2_t']
        v_t = state_ops.assign(v, v * coefficients['beta_2_t'] + v_scaled_g_values, use_locking=self._use_locking)

        denorm = (math_ops.sqrt(v_t) / math_ops.sqrt(coefficients['bias_correction2'])) + coefficients['epsilon']
        step_size = coefficients['lr'] / coefficients['bias_correction1']

        if self.nesterov:
            perturb = (coefficients['beta_1_t'] * m_t + coefficients['one_minus_beta_1_t'] * grad) / denorm
        else:
            perturb = m_t / denorm

        # Projection
        wd_ratio = 1
        if len(var.shape) > 1:
            perturb, wd_ratio = self._projection(var, grad, perturb, coefficients['delta'], coefficients['wd_ratio'], coefficients['epsilon'])

        # Weight decay

        if self.weight_decay > 0:
            var = state_ops.assign(var, var * (1 - coefficients['lr'] * coefficients['weight_decay'] * wd_ratio), use_locking=self._use_locking)

        var_update = state_ops.assign_sub(var, step_size * perturb, use_locking=self._use_locking)

        return control_flow_ops.group(*[var_update, m_t, v_t])


    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):

        var_device, var_dtype = var.device, var.dtype.base_dtype
        coefficients = ((apply_state or {}).get((var_device, var_dtype))
                        or self._fallback_apply_state(var_device, var_dtype))
        """
        Adam
        """
        # m_t = beta1 * m + (1 - beta1) * g_t
        m = self.get_slot(var, 'm')
        m_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
        m_t = state_ops.assign(m, m * coefficients['beta_1_t'],
                               use_locking=self._use_locking)
        with ops.control_dependencies([m_t]):
            m_t = self._resource_scatter_add(m, indices, m_scaled_g_values)

        # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
        v = self.get_slot(var, 'v')
        v_scaled_g_values = (grad * grad) * coefficients['one_minus_beta_2_t']
        v_t = state_ops.assign(v, v * coefficients['beta_2_t'],
                               use_locking=self._use_locking)
        with ops.control_dependencies([v_t]):
            v_t = self._resource_scatter_add(v, indices, v_scaled_g_values)

        denorm = (math_ops.sqrt(v_t) / math_ops.sqrt(coefficients['bias_correction2'])) + coefficients['epsilon']
        step_size = coefficients['lr'] / coefficients['bias_correction1']

        if self.nesterov:
            p_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
            perturb = m_t * coefficients['beta_1_t']
            perturb = self._resource_scatter_add(perturb, indices, p_scaled_g_values) / denorm

        else:
            perturb = m_t / denorm

        # Projection
        wd_ratio = 1
        if len(var.shape) > 1:
            perturb, wd_ratio = self._projection(var, grad, perturb, coefficients['delta'], coefficients['wd_ratio'], coefficients['epsilon'])

        # Weight decay
        if self.weight_decay > 0:
            var = state_ops.assign(var, var * (1 - coefficients['lr'] * coefficients['weight_decay'] * wd_ratio), use_locking=self._use_locking)

        var_update = state_ops.assign_sub(var, step_size * perturb, use_locking=self._use_locking)

        return control_flow_ops.group(*[var_update, m_t, v_t])

    def _channel_view(self, x):
        return array_ops.reshape(x, shape=[x.shape[0], -1])

    def _layer_view(self, x):
        return array_ops.reshape(x, shape=[1, -1])

    def _cosine_similarity(self, x, y, eps, view_func):
        x = view_func(x)
        y = view_func(y)

        x_norm = math_ops.euclidean_norm(x, axis=-1) + eps
        y_norm = math_ops.euclidean_norm(y, axis=-1) + eps
        dot = math_ops.reduce_sum(x * y, axis=-1)

        return math_ops.abs(dot) / x_norm / y_norm

    def _projection(self, var, grad, perturb, delta, wd_ratio, eps):
        # channel_view
        cosine_sim = self._cosine_similarity(grad, var, eps, self._channel_view)
        cosine_max = math_ops.reduce_max(cosine_sim)
        compare_val = delta / math_ops.sqrt(math_ops.cast(self._channel_view(var).shape[-1], dtype=delta.dtype))

        perturb, wd = control_flow_ops.cond(pred=cosine_max < compare_val,
                                            true_fn=lambda : self.channel_true_fn(var, perturb, wd_ratio, eps),
                                            false_fn=lambda : self.channel_false_fn(var, grad, perturb, delta, wd_ratio, eps))

        return perturb, wd

    def channel_true_fn(self, var, perturb, wd_ratio, eps):
        expand_size = [-1] + [1] * (len(var.shape) - 1)
        var_n = var / (array_ops.reshape(math_ops.euclidean_norm(self._channel_view(var), axis=-1), shape=expand_size) + eps)
        perturb -= var_n * array_ops.reshape(math_ops.reduce_sum(self._channel_view(var_n * perturb), axis=-1), shape=expand_size)
        wd = wd_ratio

        return perturb, wd

    def channel_false_fn(self, var, grad, perturb, delta, wd_ratio, eps):
        cosine_sim = self._cosine_similarity(grad, var, eps, self._layer_view)
        cosine_max = math_ops.reduce_max(cosine_sim)
        compare_val = delta / math_ops.sqrt(math_ops.cast(self._layer_view(var).shape[-1], dtype=delta.dtype))

        perturb, wd = control_flow_ops.cond(cosine_max < compare_val,
                                              true_fn=lambda : self.layer_true_fn(var, perturb, wd_ratio, eps),
                                              false_fn=lambda : self.identity_fn(perturb))

        return perturb, wd

    def layer_true_fn(self, var, perturb, wd_ratio, eps):
        expand_size = [-1] + [1] * (len(var.shape) - 1)
        var_n = var / (array_ops.reshape(math_ops.euclidean_norm(self._layer_view(var), axis=-1), shape=expand_size) + eps)
        perturb -= var_n * array_ops.reshape(math_ops.reduce_sum(self._layer_view(var_n * perturb), axis=-1), shape=expand_size)
        wd = wd_ratio

        return perturb, wd

    def identity_fn(self, perturb):
        wd = 1.0

        return perturb, wd

    def get_config(self):
        config = super(AdamP, self).get_config()
        config.update({
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'beta_1': self._serialize_hyperparameter('beta_1'),
            'beta_2': self._serialize_hyperparameter('beta_2'),
            'delta': self._serialize_hyperparameter('delta'),
            'wd_ratio': self._serialize_hyperparameter('wd_ratio'),
            'epsilon': self.epsilon,
            'weight_decay': self.weight_decay,
            'nesterov': self.nesterov
        })
        return 

## define model

LinearWarmupLRScheduler 하는 이유
- SGD는 다른 optimizer 대비 learning rate 값에 매우 민감
  - learning rate를 잘 setting 해야 성능이 좋게 나옴 (Adam보다 더 좋게 나오기도 함)
- 따라서 optimizer와 함께 learning rate도 tuning 하는 게 원래는 좋음
- 그러나 학습 속도가 너무 느려지는 단점

warmup을 하기 어려운 상황이면?
- Rectified Adam으로 먼저 테스트 해 보고, optimizer는 조절해도 거의 결과 비슷하게 나오니, 모델링 부분을 업데이트 해 보기
- Rectified Adam에도 tuning 할 수 있는 요소 많음
  - https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/RectifiedAdam

In [ ]:
n_class = 10
max_epoch = 50

with mirrored_strategy.scope():
    # channel : rgb가 없고 gray니까 1
    # model = MLP(28*28*1, 128, 64, n_class)  # *args
    model = MLPWithDropout(28*28*1, 128, 64, n_class, dropout_prob=0.3)
    model_name = type(model).__name__       # MLP

    # define loss
    loss_function = tf.losses.SparseCategoricalCrossentropy()

    # define optimizer
    lr = 1e-3
    # scheduler = None
    scheduler = LinearWarmupLRScheduler(lr, 1500)  # learning rate 추가, warmup이 있으니 속도가 느리지만 안정적인 결과 
    scheduler_name = type(scheduler).__name__ if scheduler is not None else "no_scheduler"

    if scheduler is None:
        scheduler = lr

    # optimizer = tf.optimizers.Adam(learning_rate=scheduler)
    optimizer = tf.optimizers.SGD(learning_rate=scheduler)
    # optimizer = tfa.optimizers.AdamW(learning_rate=scheduler)
    # optimizer = tfa.optimizers.RectifiedAdam(learning_rate=scheduler)
    # optimizer = AdamP(learning_rate=scheduler)
    optimizer_name = type(optimizer).__name__

    model.compile(
        loss = loss_function,
        optimizer = optimizer,
        metrics = [tf.keras.metrics.Accuracy()],
    )

    # model build
    # 이 부분 생략해도 되지만 build를 해 놓으면 나중에 debugging하기 좋음 -> 권장
    # batch 1 : 임의로 설정
    model.build((1, 28*28*1))

# 만약 build 안 하고 summary 하면 build, fit을 하거나 input shape를 넣으라고 경고 뜸
# fit은 학습이기 때문에 무거운 감이 있고 빠르게 하기 위해 build 선호
model.summary()

## define logging & callbacks

In [ ]:
log_interval = 100
run_name = f'{datetime.now()}-{model.name}-{optimizer_name}-optim-{lr}-lr-with-{scheduler_name}'

run_dirname = 'dnn-tutorial-fashion-mnist-runs-tf'

# 경로에 'run'이라는 폴더를 만들고, run_dirname에 run_name 생성
log_dir = os.path.join(drive_project_root, "runs", run_dirname, run_name)

In [ ]:
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir, update_freq=log_interval
)

## wandb setup

- https://docs.wandb.ai/guides/integrations/tensorflow
- sync_tensorboard=True : tensorflow에 적혀있는 걸 wandb에 업로드

In [ ]:
project_name = 'fastcampus_fashion_mnist_tutorials_tf'
run_tags = {project_name}   # project가 많으면 이렇게 tag명으로 관리

wandb.init(
    project=project_name, 
    name=run_name, 
    tags=run_tags, 
    config={
        "lr": lr,
        "model_name": model_name,
        "optimizer_name": optimizer_name,
        "scheduler_name": scheduler_name,
    },
    reinit=True,
    sync_tensorboard=True
    )

early stopping
- https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
  - view source code 클릭해서 참고하고 customize (세부 조정) 가능
- patience=5 : improve가 되지 않아도 5 step동안 기다리겠다는 의미

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, verbose=True)

경로 잘 찾고 있는지 확인

In [ ]:
! ls /content/drive/MyDrive/\#fastcampus/runs/

In [ ]:
# tensorboard load하기 : load extension
%load_ext tensorboard

# 경로 지정 : terminal 문법이기 때문에 #을 # 그대로 인지하려면 앞에 '\' 써줘야 함
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/

model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=max_epoch,
    # callbacks=[tb_callback] # callback은 여러 개를 넣을 수 있기 때문에 list 형태로 지정함
    callbacks=[tb_callback, early_stop_callback]
)

## model testing

In [ ]:
model.evaluate(test_dataloader)

auc curve

In [ ]:
test_labels_list = []
test_preds_list = []
test_outputs_list = []

for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc='testing')):
    with mirrored_strategy.scope():
        test_outputs = model(test_images)
    test_preds = tf.argmax(test_outputs, 1)

    final_outs = test_outputs.numpy()
    test_outputs_list.extend(final_outs)

    test_preds_list.extend(test_preds.numpy())
    test_labels_list.extend(test_labels.numpy())

test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

test_accuracy = np.mean(test_preds_list == test_labels_list)
print(f'\nacc: {test_accuracy*100}%')

roc curve

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}
n_class = 10

for i in range(n_class):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

In [ ]:
fpr

plot

In [ ]:
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle="--", label=f"Class {i} vs Rest") 

plt.title("Multi-class ROC Curve")
plt.xlabel("Flase Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="best")
plt.show()

auc score
- multi class이기 때문에 multi_class, average option 안 넣어주면 error 발생

In [ ]:
auc_score = roc_auc_score(test_labels_list, test_outputs_list, multi_class="ovo", average="macro")

In [ ]:
print(f'auc score : {auc_score*100}')